In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import math
import numpy as np

# from helpers import plot_basketball_court

locations = pd.read_csv('raw-data/train_locs.csv')
play_by_play = pd.read_csv('raw-data/train_pbp.csv')

In [2]:
#Combine CSV data into one large table
merged_df = pd.merge(play_by_play, locations, on='id', how='inner')

print(merged_df)

             id  is_oreb           team     opponent conference  \
0           2-2        0       Kentucky         Duke        SEC   
1           2-2        0       Kentucky         Duke        SEC   
2           2-2        0       Kentucky         Duke        SEC   
3           2-2        0       Kentucky         Duke        SEC   
4           2-2        0       Kentucky         Duke        SEC   
...         ...      ...            ...          ...        ...   
308045  5020-52        0  San Diego St.  Connecticut        MWC   
308046  5020-52        0  San Diego St.  Connecticut        MWC   
308047  5020-52        0  San Diego St.  Connecticut        MWC   
308048  5020-52        0  San Diego St.  Connecticut        MWC   
308049  5020-52        0  San Diego St.  Connecticut        MWC   

       opp_conference    court_x    court_y annotation_code  
0                 ACC  72.807686  37.409669              d1  
1                 ACC  79.080643  31.477796              d2  
2        

In [3]:
#Convert coordinates to only reference one direction of the court
#For now lets assume that if the shooter is on the right side of the court, the right basket is the target
#Ignore half court shots+ for now
#If the shooter has x greater than 47, mirror all points
shooter_locations = locations[(locations['annotation_code'] == 's')]
merged_df_2 = pd.merge(merged_df, shooter_locations, on='id', how='inner', suffixes=('', '_shooter'))
#if merged_df_2['court_x_shooter'] > 47, then mirror court_x to be 94 - court_x
merged_df_2['court_x'] = merged_df_2.apply(lambda x: 94 - x['court_x'] if x['court_x_shooter'] > 47 else x['court_x'], axis=1)
cleaned_data = merged_df_2.drop(['court_x_shooter', 'court_y_shooter', 'annotation_code_shooter'], axis=1)
#Note - some plays don't have a shooter and get removed
print(cleaned_data)

             id  is_oreb           team     opponent conference  \
0           2-2        0       Kentucky         Duke        SEC   
1           2-2        0       Kentucky         Duke        SEC   
2           2-2        0       Kentucky         Duke        SEC   
3           2-2        0       Kentucky         Duke        SEC   
4           2-2        0       Kentucky         Duke        SEC   
...         ...      ...            ...          ...        ...   
308045  5020-52        0  San Diego St.  Connecticut        MWC   
308046  5020-52        0  San Diego St.  Connecticut        MWC   
308047  5020-52        0  San Diego St.  Connecticut        MWC   
308048  5020-52        0  San Diego St.  Connecticut        MWC   
308049  5020-52        0  San Diego St.  Connecticut        MWC   

       opp_conference    court_x    court_y annotation_code  
0                 ACC  21.192314  37.409669              d1  
1                 ACC  14.919357  31.477796              d2  
2        

In [4]:
#add a column for dtb (distance to basket)
cleaned_data['dtb'] = np.linalg.norm(cleaned_data[['court_x', 'court_y']].values - [4, 25], axis=1)
print(cleaned_data.sort_values(by=['dtb']))

             id  is_oreb            team        opponent      conference  \
50483    1194-4        1  William & Mary        Delaware        Colonial   
136781   3138-2        0       Texas A&M  South Carolina             SEC   
135335   3105-3        0  Louisiana Tech            Rice  Conference USA   
238528  4472-24        1           Texas    Oklahoma St.          Big 12   
277578   4861-3        0            Duke  North Carolina             ACC   
...         ...      ...             ...             ...             ...   
184033  4000-28        0            Duke            Iowa             ACC   
263521  4717-40        0       Creighton      St. John's         BigEast   
263527  4717-40        0       Creighton      St. John's         BigEast   
184036  4000-28        0            Duke            Iowa             ACC   
263524  4717-40        0       Creighton      St. John's         BigEast   

        opp_conference    court_x    court_y annotation_code        dtb  
50483        

In [5]:
# Add rankings
cleaned_data['dtb_rank'] = cleaned_data.groupby(['id'])['dtb'].rank(method="first").astype(int)
cleaned_data['team'] = np.where(cleaned_data['annotation_code'].str.contains('d'), "Defense", "Offense")
cleaned_data['dtb_team_rank'] = cleaned_data.groupby(['id', 'team'])['dtb'].rank(method="first").astype(int)

In [6]:
# Pivot the dataframe for each play
pivot_df = cleaned_data.pivot_table(index=['id', 'is_oreb'], columns=['annotation_code'], values=['court_x', 'court_y', 'dtb'])
pivot_df.columns = ['{}_{}'.format(col[0], col[1]) for col in pivot_df.columns]
pivot_df = pivot_df.reset_index()

pivot_df_2 = cleaned_data.pivot_table(index=['id', 'is_oreb'], columns=['dtb_rank'], values=['dtb'])
pivot_df_2.columns = ['{}_{}'.format(col[0], col[1]) for col in pivot_df_2.columns]
pivot_df_2 = pivot_df_2.reset_index()

pivot_df_3 = cleaned_data.pivot_table(index=['id', 'is_oreb'], columns=['dtb_team_rank', 'team'], values=['dtb'])
pivot_df_3.columns = ['{}_{}_{}'.format(col[0], col[1], col[2]) for col in pivot_df_3.columns]
pivot_df_3 = pivot_df_3.reset_index()

pivot_df = pd.merge(pivot_df, pivot_df_2, on=['id', 'is_oreb'], how='inner')
pivot_df = pd.merge(pivot_df, pivot_df_3, on=['id', 'is_oreb'], how='inner')


In [7]:
#Add some aggregate columns
pivot_df['total_offense_dtb'] = pivot_df[["dtb_t1", "dtb_t2", "dtb_t3", "dtb_t4", "dtb_s"]].sum(axis=1)
pivot_df['total_defense_dtb'] = pivot_df[["dtb_d1", "dtb_d2", "dtb_d3", "dtb_d4", "dtb_d5"]].sum(axis=1)
pivot_df['top2_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense"]].sum(axis=1)
pivot_df['top3_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense"]].sum(axis=1)
pivot_df['top4_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense"]].sum(axis=1)
pivot_df['top2_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense"]].sum(axis=1)
pivot_df['top3_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense"]].sum(axis=1)
pivot_df['top4_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense"]].sum(axis=1)
pivot_df['top1_any_dtb'] = pivot_df[["dtb_1"]].sum(axis=1)
pivot_df['top2_any_dtb'] = pivot_df[["dtb_1", "dtb_2"]].sum(axis=1)
pivot_df['top3_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3"]].sum(axis=1)
pivot_df['top4_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4"]].sum(axis=1)
pivot_df['top5_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5"]].sum(axis=1)
pivot_df['top6_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6"]].sum(axis=1)
pivot_df['top7_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7"]].sum(axis=1)
pivot_df['top8_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8"]].sum(axis=1)
pivot_df['top9_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8", "dtb_9"]].sum(axis=1)
count_less_than_5 = lambda row: sum(row < 5)
pivot_df['count_less_than_5'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8", "dtb_9", 'dtb_10']].iloc[:, 1:].apply(count_less_than_5, axis=1)
pivot_df['count_less_than_5_offense'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense", "dtb_5_Offense"]].iloc[:, 1:].apply(count_less_than_5, axis=1)
pivot_df['count_less_than_5_defense'] = pivot_df[["dtb_1_Defense", "dtb_2_Defense", "dtb_3_Defense", "dtb_4_Defense", "dtb_5_Defense"]].iloc[:, 1:].apply(count_less_than_5, axis=1)
count_less_than_10 = lambda row: sum(row < 5)
pivot_df['count_less_than_10'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8", "dtb_9", 'dtb_10']].iloc[:, 1:].apply(count_less_than_10, axis=1)
pivot_df['count_less_than_10_offense'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense", "dtb_5_Offense"]].iloc[:, 1:].apply(count_less_than_10, axis=1)
pivot_df['count_less_than_10_defense'] = pivot_df[["dtb_1_Defense", "dtb_2_Defense", "dtb_3_Defense", "dtb_4_Defense", "dtb_5_Defense"]].iloc[:, 1:].apply(count_less_than_10, axis=1)


print(pivot_df)

          id  is_oreb  court_x_d1  court_x_d2  court_x_d3  court_x_d4  \
0      10-10        0   16.757017   20.172220   13.667048   11.552852   
1      10-12        0    7.229075    7.038597    9.324267   19.419485   
2      10-13        0   26.028072    8.849291   14.908529    8.849291   
3       10-3        0   16.427338   12.561111   20.535393    8.937342   
4       10-6        0   21.055481   34.011315   12.042747   15.140892   
...      ...      ...         ...         ...         ...         ...   
30800  998-6        0    6.761558   19.689568    5.915878   11.836204   
30801  999-1        0   19.336079   23.806515    8.220410    9.670299   
30802  999-3        0   12.275106    6.344312   17.065397   19.574608   
30803  999-4        0    7.416399    8.556974   22.243458   22.927847   
30804  999-5        0    6.546554    6.755028   20.097206   20.514160   

       court_x_d5  court_x_s  court_x_t1  court_x_t2  ...  top6_any_dtb  \
0       12.853857  27.490582   18.545910   28.30

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

X = pivot_df.drop(['id', 'is_oreb'], axis=1)
Y = pivot_df['is_oreb']

# Create a SimpleImputer instance to replace NaN values with the mean of the column
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
X = imputer.fit_transform(X)
X = scaler.fit_transform(X)

print(X.shape)

(30805, 70)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split



training_images, testing_images, training_labels, testing_labels = train_test_split(X,
                                                                                    Y,
                                                                                    test_size=0.2,
                                                                                    shuffle=True,
                                                                                    random_state=0)
model = Sequential()
model.add(Dense(64, input_shape=(70,)))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=20, batch_size=8, validation_data=(testing_images, testing_labels), verbose=1)


Epoch 1/20


2023-07-20 16:48:52.453305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3081/3081 [==============================] - ETA: 0s - loss: 0.5976 - accuracy: 0.7081

2023-07-20 16:49:24.272138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3081/3081 [==============================] - 35s 11ms/step - loss: 0.5976 - accuracy: 0.7081 - val_loss: 0.5884 - val_accuracy: 0.7142
Epoch 2/20
3081/3081 [==============================] - 35s 11ms/step - loss: 0.5875 - accuracy: 0.7181 - val_loss: 0.5871 - val_accuracy: 0.7143
Epoch 3/20
3081/3081 [==============================] - 34s 11ms/step - loss: 0.5867 - accuracy: 0.7184 - val_loss: 0.5884 - val_accuracy: 0.7132
Epoch 4/20
3081/3081 [==============================] - 35s 11ms/step - loss: 0.5862 - accuracy: 0.7186 - val_loss: 0.5897 - val_accuracy: 0.7145
Epoch 5/20
3081/3081 [==============================] - 34s 11ms/step - loss: 0.5855 - accuracy: 0.7188 - val_loss: 0.5857 - val_accuracy: 0.7148
Epoch 6/20
3078/3081 [============================>.] - ETA: 0s - loss: 0.5855 - accuracy: 0.7184

In [ ]:
# from helpers import plot_basketball_court
locations = pd.read_csv('raw-data/test_locs.csv')
play_by_play = pd.read_csv('raw-data/test_pbp.csv')
print(play_by_play.shape)
#Feels like its cheating the problem at hand to predict based on teams or conferences
play_by_play = play_by_play.drop(['team', 'opponent', 'conference', 'opp_conference'], axis=1)
#Combine CSV data into one large table
merged_df = pd.merge(play_by_play, locations, on='id', how='left')
print(locations.shape)
print(play_by_play.shape)
#If the shooter has x greater than 47, mirror all points
shooter_locations = locations[(locations['annotation_code'] == 's')]
merged_df_2 = pd.merge(merged_df, shooter_locations, on='id', how='left', suffixes=('', '_shooter'))
#if merged_df_2['court_x_shooter'] > 47, then mirror court_x to be 94 - court_x
merged_df_2['court_x'] = merged_df_2.apply(lambda x: 94 - x['court_x'] if x['court_x_shooter'] > 47 else x['court_x'], axis=1)
cleaned_data = merged_df_2.drop(['court_x_shooter', 'court_y_shooter', 'annotation_code_shooter'], axis=1)
#add a column for dtb (distance to basket)
cleaned_data['dtb'] = np.linalg.norm(cleaned_data[['court_x', 'court_y']].values - [4, 25], axis=1)
# Add rankings
cleaned_data['dtb_rank'] = cleaned_data.groupby(['id'])['dtb'].rank(method="first").astype(int)
cleaned_data['team'] = np.where(cleaned_data['annotation_code'].str.contains('d'), "Defense", "Offense")
cleaned_data['dtb_team_rank'] = cleaned_data.groupby(['id', 'team'])['dtb'].rank(method="first").astype(int)
# Pivot the dataframe for each play
pivot_df = cleaned_data.pivot_table(index=['id'], columns=['annotation_code'], values=['court_x', 'court_y', 'dtb'])
pivot_df.columns = ['{}_{}'.format(col[0], col[1]) for col in pivot_df.columns]
pivot_df = pivot_df.reset_index()
pivot_df_2 = cleaned_data.pivot_table(index=['id'], columns=['dtb_rank'], values=['dtb'])
pivot_df_2.columns = ['{}_{}'.format(col[0], col[1]) for col in pivot_df_2.columns]
pivot_df_2 = pivot_df_2.reset_index()
pivot_df_3 = cleaned_data.pivot_table(index=['id'], columns=['dtb_team_rank', 'team'], values=['dtb'])
pivot_df_3.columns = ['{}_{}_{}'.format(col[0], col[1], col[2]) for col in pivot_df_3.columns]
pivot_df_3 = pivot_df_3.reset_index()
pivot_df = pd.merge(pivot_df, pivot_df_2, on=['id'], how='left')
pivot_df = pd.merge(pivot_df, pivot_df_3, on=['id'], how='left')
#Add some aggregate columns
pivot_df['total_offense_dtb'] = pivot_df[["dtb_t1", "dtb_t2", "dtb_t3", "dtb_t4", "dtb_s"]].sum(axis=1)
pivot_df['total_defense_dtb'] = pivot_df[["dtb_d1", "dtb_d2", "dtb_d3", "dtb_d4", "dtb_d5"]].sum(axis=1)
pivot_df['top2_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense"]].sum(axis=1)
pivot_df['top3_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense"]].sum(axis=1)
pivot_df['top4_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense"]].sum(axis=1)
pivot_df['top2_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense"]].sum(axis=1)
pivot_df['top3_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense"]].sum(axis=1)
pivot_df['top4_offense_dtb'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense"]].sum(axis=1)
pivot_df['top1_any_dtb'] = pivot_df[["dtb_1"]].sum(axis=1)
pivot_df['top2_any_dtb'] = pivot_df[["dtb_1", "dtb_2"]].sum(axis=1)
pivot_df['top3_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3"]].sum(axis=1)
pivot_df['top4_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4"]].sum(axis=1)
pivot_df['top5_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5"]].sum(axis=1)
pivot_df['top6_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6"]].sum(axis=1)
pivot_df['top7_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7"]].sum(axis=1)
pivot_df['top8_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8"]].sum(axis=1)
pivot_df['top9_any_dtb'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8", "dtb_9"]].sum(axis=1)
count_less_than_5 = lambda row: sum(row < 5)
pivot_df['count_less_than_5'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8", "dtb_9", 'dtb_10']].iloc[:, 1:].apply(count_less_than_5, axis=1)
pivot_df['count_less_than_5_offense'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense", "dtb_5_Offense"]].iloc[:, 1:].apply(count_less_than_5, axis=1)
pivot_df['count_less_than_5_defense'] = pivot_df[["dtb_1_Defense", "dtb_2_Defense", "dtb_3_Defense", "dtb_4_Defense", "dtb_5_Defense"]].iloc[:, 1:].apply(count_less_than_5, axis=1)
count_less_than_10 = lambda row: sum(row < 5)
pivot_df['count_less_than_10'] = pivot_df[["dtb_1", "dtb_2", "dtb_3", "dtb_4", "dtb_5", "dtb_6", "dtb_7", "dtb_8", "dtb_9", 'dtb_10']].iloc[:, 1:].apply(count_less_than_10, axis=1)
pivot_df['count_less_than_10_offense'] = pivot_df[["dtb_1_Offense", "dtb_2_Offense", "dtb_3_Offense", "dtb_4_Offense", "dtb_5_Offense"]].iloc[:, 1:].apply(count_less_than_10, axis=1)
pivot_df['count_less_than_10_defense'] = pivot_df[["dtb_1_Defense", "dtb_2_Defense", "dtb_3_Defense", "dtb_4_Defense", "dtb_5_Defense"]].iloc[:, 1:].apply(count_less_than_10, axis=1)

In [ ]:
X = pivot_df.drop(['id'], axis=1)

# Create a SimpleImputer instance to replace NaN values with the mean of the column
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
X = imputer.fit_transform(X)
X = scaler.fit_transform(X)

Yhat = model.predict(X)

result_pivot_df = pivot_df
result_pivot_df['pred'] = Yhat

result_pivot_df['pred'] = result_pivot_df['pred'].apply(lambda x: round(x, 3))

# Print the resulting DataFrame with 'id' and 'Yhat' columns
print(result_pivot_df[['id', 'pred']])
result_pivot_df[['id', 'pred']].to_csv('entry1.csv', encoding='utf-8', index=False)